<a href="https://colab.research.google.com/github/ksm999/QuantumComputing/blob/main/Steregraphic_Projection_Otto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Introduction:
import os

import numpy as np
import pygame
from PIL import Image

In [26]:
file_path = '/content/drive/MyDrive/Stereo'

In [20]:
#!user/bin/env python
# -*- coding:utf-8 -*-
"""Stereographic-Projection-of-Otto:
@File: main.py
@Brief: Various forms of Otto are obtained by spherical projection.
@Author: Golevka2001<gol3vka@163.com>
@Created Date: 2022/11/29
@Last Modified Date: 2022/12/02
"""

# --------------- 매개변수 부분 --------------- #

# 원본 이미지 경로：
path_img = file_path + './otto.png'
# 투영 이미지 출력 경로
path_proj = file_path + './toot.png'

# 투영 이미지 출력 크기 (단위: 픽셀)
w_proj = 400
h_proj = 300

# Offset(단위 : 백분율)
# 비고 : 출력된 투영 이미지의 중심 위치를 조정
offset_hor = 0  # Horizontal offset (positive to right）
offset_ver = 0.4  # Vertical offset (positive downward)

# Scaling multiple
scale = 1.5

# Angle of Rotation of Coordinate axis (좌표축의 회전 각도):
# Note : Rotation is to obtain different spherical projection conditions (reason/chestnut head)
# 회전은 서로 다른 구면 투영 상황을 얻기 위한 것이다. (밤머리)
alpha = 0 * np.pi / 180  # Angle of Rotation aroun the x-axis
beta = -5 * np.pi / 180  # Rotation angle around y-axis (around 150° to get chestnut head)
gamma = 0 * np.pi / 180  # angle of rotation about z axis

# --------------- 실현 --------------- #

In [21]:
def get_point_on_sphere(point: np.ndarray, r: float) -> np.ndarray:
    """ Calculate the coordinates of the intersection point P between a point Q on the z=0 plane
    and a projection point D on the sphere

    Args:
        point (np.ndarray): point Q coordinate
        r (float): sphere radius

    Returns:
        np.ndarray: Coordinates of the intersection point P on the sphere
    """
    [x, y, z] = point
    k = 2 * r**2 / (x**2 + y**2 + r**2)
    # Derivation and simplification of the obtained coefficients (see README.md for the derivation process)
    return np.array([k * x, k * y, (k - 1) * r], dtype=np.float32)


def axis_rotate(point: np.ndarray, rot_mat: np.ndarray) -> np.ndarray:
    """Calculate the change of point P coordinate after rotation of coordinate system

    Args:
        point (np.ndarray): point P coordinate
        rot_mat (np.ndarray): Rotation matrix (see README.md for derivation)

    Returns:
        np.ndarray: transformed point P coordinate
    """
    return np.dot(rot_mat, point)


def get_pix_on_img(point: np.ndarray, r: float, h_img: int,
                   w_img: int) -> tuple:
    """The inverse process of spherical projection, calculating the coordinates of a point P on the sphere on the original image.

    Args:
        point (np.ndarray): point P coordinate
        r (float): sphere radius
        h_img (int): original image height
        w_img (int): original image width

    Returns:
        tuple: Pixel coordinates corresponding to the original image
    """
    [x, y, z] = point
    if z > r:
        z = r
    row = np.arccos(z / r) / np.pi
    col = np.arctan2(y, x) / 2 / np.pi + 0.5  # Adding 0.5 is moving the center of the image to the plane y=0.
    # The coordinate range is restored to the original image size:
    row = round(row * h_img) % h_img
    col = round(col * w_img) % w_img
    return (row, col)


def projection(pix_proj: tuple, r: float, h_img: int, w_img: int, h_proj: int,
               w_proj: int) -> tuple:
    """spherical projection

    Args:
        pix_proj (tuple): pixel coordinates on projected images
        r (float): sphere radius
        h_img (int): original image height
        w_img (int): original image width
        h_proj (int): projected image height
        w_proj (int): projection width

    Returns:
        tuple: Pixel coordinates corresponding to the original image
    """
    # The pixel point coordinate on the projection image is converted to 3D coordinate:
    (row, col) = pix_proj
    x = row + (offset_ver - 0.5) * h_proj
    y = col + (offset_hor - 0.5) * w_proj
    z = 0
    Q = np.array([x, y, z], dtype=np.float32)
    P = get_point_on_sphere(Q, r)
    P = axis_rotate(P, rot_mat)
    return get_pix_on_img(P, r, h_img, w_img)


In [ ]:
!pip install pygame

In [ ]:
from pygame import *

In [29]:
import os
print(os.getcwd())

/content/drive/MyDrive


In [31]:
if __name__ == '__main__':
    '''
    pygame.mixer.init()
    pygame.mixer.music.load(file_path + 'bgm.wav')
    pygame.mixer.music.set_volume(0.3)
    pygame.mixer.music.play()
    '''

    path_img = os.path.join(os.path.abspath(os.path.dirname(__file__)),
                            path_img)
    path_proj = os.path.join(os.path.abspath(os.path.dirname(__file__)),
                             path_proj)

    arr_img = np.array(Image.open(path_img))
    arr_proj = np.zeros((h_proj, w_proj, 3), dtype=np.uint8)

    h_img = arr_img.shape[0]
    w_img = arr_img.shape[1]

    r = min(h_proj, w_proj) / 10 * scale  # The radius of the sphere affects how much content is presented on the projected image.

    # It's always automatically formatted to look so ugly.
    rot_mat = np.array([[
        np.cos(gamma) * np.cos(beta),
        np.cos(gamma) * np.sin(beta) * np.sin(alpha) -
        np.sin(gamma) * np.cos(alpha),
        np.cos(gamma) * np.sin(beta) * np.cos(alpha) +
        np.sin(gamma) * np.sin(alpha)
    ],
                        [
                            np.sin(gamma) * np.cos(beta),
                            np.sin(gamma) * np.sin(beta) * np.sin(alpha) +
                            np.cos(gamma) * np.cos(alpha),
                            np.sin(gamma) * np.sin(beta) * np.cos(alpha) -
                            np.cos(gamma) * np.sin(alpha)
                        ],
                        [
                            -np.sin(beta),
                            np.cos(beta) * np.sin(alpha),
                            np.cos(beta) * np.cos(alpha)
                        ]])

    # That is, lay the target image on the xy plane (the center of the image is O, so pay attention to the range of coordinates)
    # Through each pixel point, the coordinates of the intersections on the sphere are obtained, and the inverse transformation of the spherical projection corresponds to the pixels on the original image.
    for pix_proj in np.ndindex(arr_proj.shape[:2]):
        pix_img = projection(pix_proj, r, h_img, w_img, h_proj, w_proj)
        arr_proj[pix_proj] = arr_img[pix_img]

    pygame.mixer.music.unload()
    pygame.mixer.music.load(file_path + 'dududu.mp3')
    pygame.mixer.music.set_volume(0.8)
    pygame.mixer.music.play()

    Image.fromarray(arr_proj).show()  # annotate this line without pop-up
    # Image.fromarray(arr_proj).save(path_proj)  # annotate this line without outputting the file


NameError: ignored

In [ ]:
#!user/bin/env python
# -*- coding:utf-8 -*-
"""Stereographic-Projection-of-Otto:
@File: main.py
@Brief: Various forms of Otto are obtained by spherical projection.
@Author: Golevka2001<gol3vka@163.com>
@Created Date: 2022/11/29
@Last Modified Date: 2022/12/02
"""

# 开导：
import os

import numpy as np
import pygame
from PIL import Image

# --------------- 参数部分 --------------- #

# 原始图像路径：
path_img = './otto.png'
# 投影图像输出路径：
path_proj = './toot.png'

# 投影图像输出尺寸（单位：像素）：
w_proj = 400
h_proj = 300

# 偏移量（单位：百分比）：
# 注：用于调整输出的投影图像中心在投影平面上的位置：
offset_hor = 0  # 水平方向偏移量（向右为正）
offset_ver = 0.4  # 垂直方向偏移量（向下为正）

# 缩放倍数
scale = 1.5

# 坐标轴的旋转角度：
# 注：旋转是为了得到不同的球面投影情况（说的道理/栗子头）
alpha = 0 * np.pi / 180  # 绕x轴旋转角度
beta = -5 * np.pi / 180  # 绕y轴旋转角度（150°左右可得到栗子头）
gamma = 0 * np.pi / 180  # 绕z轴旋转角度

# --------------- 实现 --------------- #


def get_point_on_sphere(point: np.ndarray, r: float) -> np.ndarray:
    """计算z=0平面上一点Q与投影点D连线在球面上的交点P的坐标

    Args:
        point (np.ndarray): 点Q坐标
        r (float): 球半径

    Returns:
        np.ndarray: 球面上交点P的坐标
    """
    [x, y, z] = point
    k = 2 * r**2 / (x**2 + y**2 + r**2)  # 推导、化简得到的系数（推导过程见README.md）
    return np.array([k * x, k * y, (k - 1) * r], dtype=np.float32)


def axis_rotate(point: np.ndarray, rot_mat: np.ndarray) -> np.ndarray:
    """计算坐标系旋转后，点P坐标的变化

    Args:
        point (np.ndarray): 点P坐标
        rot_mat (np.ndarray): 旋转矩阵（推导过程见README.md）

    Returns:
        np.ndarray: 变换后的点P坐标
    """
    return np.dot(rot_mat, point)


def get_pix_on_img(point: np.ndarray, r: float, h_img: int,
                   w_img: int) -> tuple:
    """球面投影的逆过程，计算球面上一点P在原图像上的坐标

    Args:
        point (np.ndarray): 点P坐标
        r (float): 球半径
        h_img (int): 原始图像高度
        w_img (int): 原始图像宽度

    Returns:
        tuple: 对应在原始图像上的像素点坐标
    """
    [x, y, z] = point
    if z > r:
        z = r
    row = np.arccos(z / r) / np.pi
    col = np.arctan2(y, x) / 2 / np.pi + 0.5  # 加0.5是把图像中心移到平面y=0处
    # 坐标范围恢复到原始图像的尺寸：
    row = round(row * h_img) % h_img
    col = round(col * w_img) % w_img
    return (row, col)


def projection(pix_proj: tuple, r: float, h_img: int, w_img: int, h_proj: int,
               w_proj: int) -> tuple:
    """球极投影

    Args:
        pix_proj (tuple): 投影图像上的像素点坐标
        r (float): 球半径
        h_img (int): 原始图像高度
        w_img (int): 原始图像宽度
        h_proj (int): 投影图像高度
        w_proj (int): 投影图像宽度

    Returns:
        tuple: 对应在原始图像上的像素点坐标
    """
    # 投影图像上像素点坐标转为三维坐标：
    (row, col) = pix_proj
    x = row + (offset_ver - 0.5) * h_proj
    y = col + (offset_hor - 0.5) * w_proj
    z = 0
    Q = np.array([x, y, z], dtype=np.float32)
    P = get_point_on_sphere(Q, r)
    P = axis_rotate(P, rot_mat)
    return get_pix_on_img(P, r, h_img, w_img)


if __name__ == '__main__':
    pygame.mixer.init()
    pygame.mixer.music.load('bgm.wav')
    pygame.mixer.music.set_volume(0.3)
    pygame.mixer.music.play()

    path_img = os.path.join(os.path.abspath(os.path.dirname(__file__)),
                            path_img)
    path_proj = os.path.join(os.path.abspath(os.path.dirname(__file__)),
                             path_proj)

    arr_img = np.array(Image.open(path_img))
    arr_proj = np.zeros((h_proj, w_proj, 3), dtype=np.uint8)

    h_img = arr_img.shape[0]
    w_img = arr_img.shape[1]

    r = min(h_proj, w_proj) / 10 * scale  # 球的半径会影响到投影图像上呈现内容的多少

    # 这个总是被自动格式化成这样很丑
    rot_mat = np.array([[
        np.cos(gamma) * np.cos(beta),
        np.cos(gamma) * np.sin(beta) * np.sin(alpha) -
        np.sin(gamma) * np.cos(alpha),
        np.cos(gamma) * np.sin(beta) * np.cos(alpha) +
        np.sin(gamma) * np.sin(alpha)
    ],
                        [
                            np.sin(gamma) * np.cos(beta),
                            np.sin(gamma) * np.sin(beta) * np.sin(alpha) +
                            np.cos(gamma) * np.cos(alpha),
                            np.sin(gamma) * np.sin(beta) * np.cos(alpha) -
                            np.cos(gamma) * np.sin(alpha)
                        ],
                        [
                            -np.sin(beta),
                            np.cos(beta) * np.sin(alpha),
                            np.cos(beta) * np.cos(alpha)
                        ]])

    # 即把目标图像平铺在xy平面上（图片中心在O，所以注意坐标的范围）
    # 遍历每一个像素点，得到在球上的交点坐标，再由球面投影的逆变换对应到原图像上的像素点
    for pix_proj in np.ndindex(arr_proj.shape[:2]):
        pix_img = projection(pix_proj, r, h_img, w_img, h_proj, w_proj)
        arr_proj[pix_proj] = arr_img[pix_img]

    pygame.mixer.music.unload()
    pygame.mixer.music.load('dududu.mp3')
    pygame.mixer.music.set_volume(0.8)
    pygame.mixer.music.play()

    Image.fromarray(arr_proj).show()  # 注释掉这行可以不弹出显示
    # Image.fromarray(arr_proj).save(path_proj)  # 注释掉这行可以不输出文件


pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: ignored